In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import os
import sys
cm_folder = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/"
src_path = os.path.join(cm_folder, "src")
sys.path.append(src_path)

In [2]:
from typing import Any

import dill
import numpy as np

from CrossValidation import cross_validation
from MIRA import MIRA
from Settings import Settings

from window_based_tagger_config import get_config
from crel_helper import get_cr_tags
from crel_processing import essay_to_crels_cv
from evaluation import evaluate_model_essay_level, get_micro_metrics, metrics_to_df
from feature_normalization import min_max_normalize_feats
from function_helpers import get_function_names
from results_procesor import ResultsProcessor
from train_parser import essay_to_crels, create_extractor_functions
from cost_functions import micro_f1_cost_plusepsilon
from train_reranker import train_model, train_instance, get_essays_for_data, evaluate_ranker
from searn_parser_breadth_first import SearnModelBreadthFirst
from causal_model_features import CausalModelType
from feature_extraction import get_features_from_probabilities
from results_procesor import ResultsProcessor
from filter_features import filter_feats

from wordtagginghelper import merge_dictionaries
from results_procesor import ResultsProcessor, __MICRO_F1__
from evaluation import add_cr_labels

from random import shuffle
from joblib import Parallel, delayed
from collections import defaultdict

In [3]:
def train_model_fold(xs_train, xs_test, name2essay, C, pa_type, max_update_items, set_cr_tags,\
                     initial_weight, max_epochs, early_stop_iters):

    mdl = MIRA(
        C=C, pa_type=pa_type, max_update_items=max_update_items, initial_weight=initial_weight)

    return train_model(mdl, xs_train=xs_train, xs_test=xs_test, name2essay=name2essay,
            max_epochs=max_epochs, early_stop_iters=early_stop_iters, set_cr_tags=set_cr_tags,
            train_instance_fn=train_instance,
            verbose=False, return_metrics=True, early_stopping=False)

def train_model_parallel(cv_folds, name2essay, C, pa_type, max_update_items, set_cr_tags, \
                         initial_weight, max_epochs=5, early_stop_iters=5, n_jobs=None):

    if n_jobs == None:
        n_jobs = len(cv_folds)
    try:
        results = Parallel(n_jobs=n_jobs)(
            delayed(train_model_fold)(train, test, name2essay, C, pa_type, max_update_items, set_cr_tags, \
                                      initial_weight, max_epochs, early_stop_iters)
            for (train, test) in cv_folds)

        f1s = []
        for tpl in results:
            best_test_f1, best_iterations, train_ys_bytag, train_pred_ys_bytag, test_ys_bytag, test_pred_ys_bytag, num_feats = tpl
            f1s.append(best_test_f1)

        return np.mean(f1s)

    except KeyboardInterrupt:
        print("Process stopped by user")

def train_model_parallel_logged(training_collection_name: str, results_processor: ResultsProcessor,
                                feat_extractors, params,
                                cv_folds, name2essay,
                                C: float, pa_type: str, max_update_items:int, set_cr_tags, \
                                initial_weight: float,  max_epochs=5, early_stop_iters=5, n_jobs=None):
    if not n_jobs or n_jobs == None:
        n_jobs = len(cv_folds)

    try:
        results = Parallel(n_jobs=n_jobs)(
            delayed(train_model_fold)(train, test, name2essay, C, pa_type, max_update_items, set_cr_tags, \
                                      initial_weight, max_epochs, early_stop_iters)
            for (train, test) in cv_folds)

        cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag = defaultdict(list), defaultdict(list)
        cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = defaultdict(list), defaultdict(list)

        f1s = []
        feats = []
        for tpl in results:
            best_test_f1, best_iterations, train_ys_bytag, train_pred_ys_bytag, test_ys_bytag, test_pred_ys_bytag, num_feats = tpl
            feats.append(num_feats)
            f1s.append(best_test_f1)

            merge_dictionaries(train_ys_bytag, cv_sent_td_ys_by_tag)
            merge_dictionaries(test_ys_bytag, cv_sent_vd_ys_by_tag)

            merge_dictionaries(train_pred_ys_bytag, cv_sent_td_predictions_by_tag)
            merge_dictionaries(test_pred_ys_bytag, cv_sent_vd_predictions_by_tag)


        ALGO = "MIRA Re-Ranker"
        validation_collection = training_collection_name.replace("_TD", "_VD")

        # extractors = list(map(lambda fn: fn.func_name, feat_extractors))
        extractors = list(feat_extractors)

        parameters = {
            "C":                    C,
            "pa_type":              pa_type,
            "loss_type":            "None - cost insens",
            "max_update_items":     max_update_items,
            "initial_weight":       initial_weight,

            "max_epochs":           max_epochs,
            "early_stopping_iters": early_stop_iters,

            "extractors":           extractors,

            # Add in number of features
            "num_feats_per_fold":   feats,
            "num_feats_MEAN":       np.mean(feats)
        }
        # add in additional parameters not passed in
        parameters.update(params)

        wd_td_objectid = results_processor.persist_results(training_collection_name,
                                                           cv_sent_td_ys_by_tag,
                                                           cv_sent_td_predictions_by_tag,
                                                           parameters, ALGO)

        wd_vd_objectid = results_processor.persist_results(validation_collection,
                                                           cv_sent_vd_ys_by_tag,
                                                           cv_sent_vd_predictions_by_tag,
                                                           parameters, ALGO)

        avg_f1 = float(results_processor.get_metric(validation_collection, wd_vd_objectid, __MICRO_F1__)["f1_score"])
        return avg_f1

    except KeyboardInterrupt:
        print("Process stopped by user")

In [5]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
CAUSAL_MODEL_TYPE = CausalModelType.CORAL_BLEACHING
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

crels_folder = "./crels/CB"
coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

MONGO_COLLECTION = "CB_PA_RE-RANKER_HYPER_PARAM_TD"
# first and second were with initial_weight set to 1.0
# thrid is with set to 0.001

config = get_config(training_folder)
results_processor = ResultsProcessor(dbname="metrics_causal_model_reranker")

train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
902 226


In [6]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)

set_cr_tags = set(cr_tags)
list(set_cr_tags)[0:10]

['Causer:50->Result:7',
 'Causer:11->Result:11',
 'Causer:6->Result:50',
 'Causer:12->Result:50',
 'Causer:11->Result:3',
 'Causer:7->Result:14',
 'Causer:7->Result:50',
 'Causer:12->Result:13',
 'Causer:13->Result:12',
 'Causer:2->Result:1']

In [7]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
name2essay = {}
for essay in all_essays:
    name2essay[essay.name] = essay

name2crels = essay_to_crels(all_essays, set_cr_tags)
assert len(name2crels) == len(all_essays)

# Train Re-Ranker

## Extract Parses from Sentence Parser

In [8]:
best_top_n = 2
min_feat_freq = 1
best_max_upd = 2 
best_max_parses = 300
best_min_prob = 0.0  # min prob of 0 seems better

In [11]:
rr_fname = "xs_rerank_" + str(best_top_n) + ".dill"
with open(os.path.join(crels_folder, rr_fname), "rb") as f:
    xs_rerank = dill.load(f)

rr_fname = "xs_rerank_test" + str(best_top_n) + ".dill"
with open(os.path.join(crels_folder, rr_fname), "rb") as f:
    xs_test_rerank = dill.load(f)
    
assert len(xs_rerank) == len(pred_tagged_essays_train),     "Wrong number of train crels"
assert len(xs_test_rerank) == len(pred_tagged_essays_test), "Wrong number of test crels"
len(xs_rerank), len(xs_test_rerank)

(902, 226)

## Prepare Features

In [12]:
best_max_parses, min_feat_freq, best_min_prob

(300, 1, 0.0)

In [13]:
%%time
xs = get_features_from_probabilities(xs_rerank, name2crels, best_max_parses, 
                                     causal_model_type=CAUSAL_MODEL_TYPE,
                                     min_feat_freq=min_feat_freq, min_prob=best_min_prob)

cv_folds_rerank = cross_validation(xs, 5)
cv_folds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_folds_rerank]

CPU times: user 1min 48s, sys: 2.39 s, total: 1min 50s
Wall time: 1min 50s


In [14]:
%%time
xs_test = get_features_from_probabilities(xs_test_rerank, name2crels, best_max_parses, 
                                          causal_model_type=CAUSAL_MODEL_TYPE,
                                          min_feat_freq=min_feat_freq, min_prob=best_min_prob)

CPU times: user 5.38 s, sys: 69.8 ms, total: 5.45 s
Wall time: 5.44 s


In [15]:
# Prepare test dataset 
  # training data comes from the test fold predictions from CV on the full training dataset
xs_train = []
for train, test in cv_folds_rerank:
    xs_train.extend(test)

# Normalize both using training data
xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train,xs_test)

### Initial Parameters

In [20]:
best_C = 0.0025       # 0.0025
C = best_C            # This needs to be a lot lower
pa_type = 1
loss_type= "ml"
max_update_items = 2  # best_max_upd - 2
initial_weight = 0.01  # was 0.01

## Train on Test Data (i.e. Held out Folds)

In [22]:
num_train = int(0.8 * len(xs_train_mm))
tmp_train_copy = list(xs_train_mm)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

### Determine Number of Training Iterations

In [24]:
train_instance

<function train_reranker.train_instance>

In [25]:
%%time
# use training data to determine number of iterations
mdl = MIRA(C=best_C, pa_type=pa_type, max_update_items=max_update_items, initial_weight=initial_weight)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(
    mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, 
    set_cr_tags=set_cr_tags, max_epochs=20, early_stop_iters=5, 
     train_instance_fn = train_instance, verbose=True, early_stopping=True)
print("Best iterations:", best_iterations)

Epoch: 0 Train Accuracy: 0.7428 Test Accuracy: 0.7391
Epoch: 1 Train Accuracy: 0.7466 Test Accuracy: 0.7296
Epoch: 2 Train Accuracy: 0.7492 Test Accuracy: 0.7254
Epoch: 3 Train Accuracy: 0.7513 Test Accuracy: 0.7252
Epoch: 4 Train Accuracy: 0.7510 Test Accuracy: 0.7260
Epoch: 5 Train Accuracy: 0.7521 Test Accuracy: 0.7300
Best Test Acc: 0.7391
Best iterations: 1
CPU times: user 2min 26s, sys: 740 ms, total: 2min 27s
Wall time: 2min 27s


In [26]:
MONGO_COLLECTION, best_iterations

('CB_PA_RE-RANKER_HYPER_PARAM_TD', 1)

## Feature Selection

In [27]:
from filter_features import filter_feats

prefixes = [
    "Prob-",
#     "CREL_Pair-",
    "Inv-",
    "num_crels",
    "Tally-",
    "CChain-",
    "CChainStats-",
    "Above-",
    "CREL_",
    "Propn_",
    "Diff_"
]
# xs_fltr_train, xs_fltr_test = filter_feats(xs_train_mm, xs_test_mm, prefixes)
assert len(prefixes) == len(set(prefixes)), "Duplicate prefixes found"

In [28]:
best_f1 = -1
current_best = []
remaining = list(prefixes)

In [30]:
MONGO_COLLECTION, best_iterations, initial_weight

('CB_PA_RE-RANKER_HYPER_PARAM_TD', 1, 0.01)

In [ ]:
%%time
params = {
    "best_top_n":      best_top_n,
    "best_max_upd":    best_max_upd,
    "best_max_parses": best_max_parses,
    "best_min_prob":   best_min_prob,
    "min_feat_freq":   min_feat_freq,
    "best_iterations": best_iterations
}

print("Starting...")
while True:
    if len(remaining) == 0:
        break
    
    f1_by_prefix = dict()
    for prefix in remaining:
        new_prefixes = current_best + [prefix]
        
        cv_filtered = []
        for tr, test in cv_folds_mm:
            x_tr,x_test = filter_feats(tr, test, new_prefixes)
            cv_filtered.append((x_tr,x_test))
        
        f1_by_prefix[prefix] = train_model_parallel_logged(
            training_collection_name=MONGO_COLLECTION, results_processor=results_processor,
            feat_extractors=new_prefixes, params=params,
            
            cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
            pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
            set_cr_tags=set_cr_tags, initial_weight=initial_weight,
            # use best iterations from above
            max_epochs=best_iterations, early_stop_iters=best_iterations
        )
        print("\t{length} feats F1: {f1:.6f} Prefixes: {prefixes}".format(
            length=len(new_prefixes), f1=f1_by_prefix[prefix], prefixes=str(new_prefixes)))
    
    best_prefix, new_best_f1 = sorted(f1_by_prefix.items(), key = lambda tpl: -tpl[1])[0]
    if new_best_f1 > best_f1:
        best_f1 = new_best_f1
        current_best.append(best_prefix)
        remaining.remove(best_prefix)
        print("{length} feats, new Best F1: {f1:.6f} Prefixes: {prefixes}".format(
            length=len(current_best), f1=best_f1, prefixes=str(current_best)))
    else:
        print("No further improvement, stopping")
        break

# 1 feats, new Best F1: 0.7430 Prefixes: ['CREL_']
# 2 feats, new Best F1: 0.7438 Prefixes: ['CREL_', 'Prob-']
# 3 feats, new Best F1: 0.7461 Prefixes: ['CREL_', 'Prob-', 'CChainStats-']
# No further improvement, stopping

# Hyper Parameter Optimization

In [16]:
import datetime
datetime.datetime.now()

datetime.datetime(2019, 6, 23, 12, 9, 36, 392557)

In [17]:
best_f1 = -1
# best_f1 = 0.7457

In [18]:
# Form a closure to simplify loop
def trn_mdl(top_n, prms, cv_filtrd):
    
    global best_f1, best_C, pa_type, best_max_upd, initial_weight
    
    n_jobs = None
    # Uses too much RAM, drop to single threaded
    if top_n > 3:        
        n_jobs = 1
    
    f1 = train_model_parallel_logged(
        training_collection_name=MONGO_COLLECTION, results_processor=results_processor,
        feat_extractors=current_best, params=prms,

        cv_folds=cv_filtrd, name2essay=name2essay, 
        C=best_C, pa_type=pa_type, max_update_items=best_max_upd, initial_weight=initial_weight,
        set_cr_tags=set_cr_tags,
        # use best iterations from above
        max_epochs=best_iterations, early_stop_iters=best_iterations,
        n_jobs=n_jobs
    )
    if f1 > best_f1:
        best_f1 = f1
        print("New Best F1: {f1:.6f}:\t{params}".format(f1=f1, params=\
                                                       str((top_n, best_C, pa_type, best_max_upd, initial_weight))))
    else:
        print("         F1: {f1:.6f}:\t{params}".format(f1=f1, params=\
                                                       str((top_n, best_C, pa_type, best_max_upd, initial_weight))))

In [25]:
param_hash = load_param_hash(client.metrics_causal_model_reranker, MONGO_COLLECTION)

for top_n in [2,5,3,1]:
    
    print("top_n", top_n)
    xs_rr, _ = load_rerank(top_n)
    xs_temp = get_features_from_probabilities(xs_rr, name2crels, best_max_parses, 
                                     causal_model_type=CAUSAL_MODEL_TYPE,
                                     min_feat_freq=min_feat_freq, min_prob=best_min_prob)
    
    cv_flds_rr = cross_validation(xs_temp, 5)
    cv_flds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_flds_rr]
    
    cv_filtered = []
    for tr, test in cv_flds_mm:
        x_tr,x_test = filter_feats(tr, test, current_best)
        cv_filtered.append((x_tr,x_test))
    
    skipped = False
    for best_C in [0.0005, 0.0025, 0.0100, 0.1]:
        for pa_type in [1,2]: # [0]
            for best_max_upd in [1]:
                for initial_weight in [0.01]:                        
                    p = {'C': best_C,
                         'best_max_parses': best_max_parses,
                         'best_min_prob': best_min_prob,
                         'best_top_n': top_n,
                         'extractors': list(current_best),
                         'initial_weight': initial_weight,
                         'max_update_items': best_max_upd,
                         'min_feat_freq': min_feat_freq,
                         'pa_type': pa_type
                    }
                    hash_p = hash_params(p)
                    if hash_p in param_hash:
                        print(".", end = '')
                        skipped = True
                        continue
                    if skipped:
                        print()
                    trn_mdl(top_n, p, cv_filtered)
                    skipped = False

# top_n, best_C, pa_type, best_max_upd, initial_weight

len(rows) 1
top_n 2
.
         F1: 0.729505:	(2, 0.0005, 2, 1, 0.01)
         F1: 0.724778:	(2, 0.0025, 1, 1, 0.01)
         F1: 0.727309:	(2, 0.0025, 2, 1, 0.01)
New Best F1: 0.732591:	(2, 0.01, 1, 1, 0.01)
         F1: 0.730584:	(2, 0.01, 2, 1, 0.01)
New Best F1: 0.734815:	(2, 0.1, 1, 1, 0.01)
         F1: 0.725671:	(2, 0.1, 2, 1, 0.01)
top_n 5
         F1: 0.727237:	(5, 0.0005, 1, 1, 0.01)
         F1: 0.722649:	(5, 0.0005, 2, 1, 0.01)
         F1: 0.718399:	(5, 0.0025, 1, 1, 0.01)
         F1: 0.720851:	(5, 0.0025, 2, 1, 0.01)
         F1: 0.724311:	(5, 0.01, 1, 1, 0.01)
         F1: 0.729129:	(5, 0.01, 2, 1, 0.01)
         F1: 0.732238:	(5, 0.1, 1, 1, 0.01)
         F1: 0.720958:	(5, 0.1, 2, 1, 0.01)
top_n 3
         F1: 0.727525:	(3, 0.0005, 1, 1, 0.01)
         F1: 0.721481:	(3, 0.0005, 2, 1, 0.01)
         F1: 0.721298:	(3, 0.0025, 1, 1, 0.01)
         F1: 0.723114:	(3, 0.0025, 2, 1, 0.01)
         F1: 0.723626:	(3, 0.01, 1, 1, 0.01)
         F1: 0.729233:	(3, 0.01, 2, 1, 0.01)

## Apply to Test Data

In [26]:
min_feat_freq, best_min_prob

(1, 0.0)

### Set Optimal Features

In [28]:
# f1_score	precision	recall	    best_top_n	C	    best_max_parses	  max_update_items	initial_weight	pa_type
# 0.737302	0.807826	0.678102	1	        0.10	300	              1	                0.01	        1
        
# # current best settings
# C=best_C, pa_type=pa_type, max_update_items=best_max_upd, initial_weight=initial_weight,
top_n = 1
best_C = 0.10
best_max_upd = 1
max_update_items = 1
initial_weight = 0.01
pa_type = 1

In [29]:
xs_rr, xs_tst_rr = load_rerank(top_n)
xs_temp = get_features_from_probabilities(xs_rr, name2crels, best_max_parses, 
                                 causal_model_type=CAUSAL_MODEL_TYPE,
                                 min_feat_freq=min_feat_freq, min_prob=best_min_prob)

# Don't need CV here, but leave it in anyways
cv_flds_rr = cross_validation(xs_temp, 5)    
xs_test = get_features_from_probabilities(xs_tst_rr, name2crels, best_max_parses, 
                                      causal_model_type=CAUSAL_MODEL_TYPE,
                                      min_feat_freq=min_feat_freq, min_prob=best_min_prob)
xs_train_tmp = []
for train, test in cv_flds_rr:
    xs_train_tmp.extend(test)

xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train_tmp, xs_test)
# Filter at the end
xs_train_mm_fltr, xs_test_mm_fltr = filter_feats(xs_train_mm, xs_test_mm, current_best) 

### Determine the Optimal Number of Training Iterations

In [30]:
# tuning dataset from training data
num_train = int(0.8 * len(xs_train_mm_fltr))
tmp_train_copy = list(xs_train_mm_fltr)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

In [31]:
%%time
mdl = MIRA(C=best_C, pa_type=pa_type,
                        max_update_items=best_max_upd, initial_weight=initial_weight)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=3, train_instance_fn = train_instance,
                                                        verbose=True,  early_stopping=True)

Epoch: 0 Train Accuracy: 0.7533 Test Accuracy: 0.7116
Epoch: 1 Train Accuracy: 0.7596 Test Accuracy: 0.7205
Epoch: 2 Train Accuracy: 0.7639 Test Accuracy: 0.7216
Epoch: 3 Train Accuracy: 0.7682 Test Accuracy: 0.7216
Epoch: 4 Train Accuracy: 0.7678 Test Accuracy: 0.7190
Epoch: 5 Train Accuracy: 0.7681 Test Accuracy: 0.7185
Best Test Acc: 0.7216
CPU times: user 1min 2s, sys: 493 ms, total: 1min 3s
Wall time: 1min 3s


In [36]:
best_iterations

3

## Run for X Iterations on the Test Dataset

In [33]:
best_C, pa_type, best_max_upd, initial_weight

(0.1, 1, 1, 0.01)

In [34]:
mdl = MIRA(C=best_C, pa_type=pa_type, 
                        max_update_items=best_max_upd, initial_weight=initial_weight)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=best_iterations, early_stop_iters=best_iterations,
    train_instance_fn = train_instance, verbose=True, early_stopping=False)

# Best Test Acc: 0.7516
best_iterations

Epoch: 0 Train Accuracy: 0.7455 Test Accuracy: 0.7479
Epoch: 1 Train Accuracy: 0.7537 Test Accuracy: 0.7554
Epoch: 2 Train Accuracy: 0.7611 Test Accuracy: 0.7509
Best Test Acc: 0.7554


3

In [35]:
sorted(best_mdl.weights.items(), key = lambda tpl: -abs(tpl[1]))[0:5]

[('CREL_Pair-Causer:11->Result:13|Causer:14->Result:50', 1.2100000000000002),
 ('Prob-prod-prob', 1.1245595918767992),
 ('CREL_Pair-Causer:14->Result:50|Causer:6->Result:14', 1.0868652481887433),
 ('CREL_Pair-Causer:3->Result:4|Causer:4->Result:50', -0.8899999999999999),
 ('CREL_Causer:1->Result:50-MIN(prob)', 0.839422169128461)]

In [37]:
test_folds = [(xs_train_mm_fltr, xs_test_mm_fltr)]

params = {
    'C': best_C,
    'best_max_parses': best_max_parses,
    'best_min_prob': best_min_prob,
    'best_top_n': top_n,
    'extractors': list(current_best),
    'initial_weight': initial_weight,
    'max_update_items': best_max_upd,
    'min_feat_freq': min_feat_freq,
    'pa_type': pa_type
}

test_f1 = train_model_parallel_logged(
        training_collection_name=MONGO_TEST_COLLECTION, results_processor=results_processor,
        feat_extractors=current_best, params=params,

        cv_folds=test_folds, 
        
        name2essay=name2essay, 
        C=best_C, pa_type=pa_type, max_update_items=best_max_upd, initial_weight=initial_weight,
        set_cr_tags=set_cr_tags,
        # use best iterations from above
        max_epochs=best_iterations, early_stop_iters=best_iterations
    )
test_f1

0.7521206409048069

## TODO

### Ideas
- ~~num-crels - add back in the logic to threshold these? But only if needed to improve results here (seemed to help essay parser)~~
- ~~Re-run with more realistic initial hyper params~~
- ~~Add in logic to store results to mongo~~
- ~~Switch back to using an initial_weight of 1~~
- Approach seems very sensitive to the initial configuration of the algorithm. However, it also seems correlated to the training data performance on the first epoch. Run the algorithm multiple times, take the model with the best training performance and use that as the final selected model to train futher.
- We need to add hyper parameter tuning
- Do we want to just remove the BEAM search from this? It makes the explanation a lot more complex. But then again, it's the only way we can really add more crels that the model wouldn't otherwise parse
- Do we use the BEAM search with some de-duping? Although we already de-dupe to some extent anyways

### Needed to Finish
- Record run on test data - needs optimal hyper parameters first